<a href="https://colab.research.google.com/github/atotev/ca683-2021-dara/blob/main/Importing_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install requests

In [36]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


In [42]:
apiKey = "defc104635dae32057dd64f95aaf4e0d"


In [43]:
link ='https://drive.google.com/file/d/1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v/view?usp=sharing'

In [46]:


downloaded = drive.CreateFile({'id':'1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v'}) 
fileMovies = 'movie_titles.csv'

downloaded.GetContentFile(fileMovies)  

movies_netflix = pd.read_csv(fileMovies, 
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['id', 'year', 'title']).set_index('id')

print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))
movies_netflix.sample(5)

#total of 17770



Shape Movie-Titles:	(17770, 2)


,year,title
id,,
1274,1984.0,At Close Range
2660,1989.0,When Harry Met Sally
10476,1932.0,Trouble in Paradise
9455,1967.0,The Happiest Millionaire
3015,2004.0,The Final Cut


In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Search for movies in the Movie DB API**
- For each movie found store genre, popularity, votes, genre and media type
- Ignore titles found with category 'People' as it would actors/directores an dnov movies

https://developers.themoviedb.org/3/search/search-movies

In [80]:
def getFirstPartMovieName(movie):
  if movie.find(":") != -1:
    return movie[0:movie.find(":")]
  if movie.find("(") != -1:
    return movie[0:movie.find("(")]
  if movie.find("/") != -1:
    return movie[0:movie.find("/")]
  return movie

def period_is_ok(release_date, year):
    if (len(release_date)>=4 and len(year) >= 4):
      release_date_int = int(float(release_date[0:4]))
      year_int = int(float(year[0:4]))
      difference = (release_date_int - year_int)
      if (difference >= -3 and difference <= 3 ):
        return True
    return False

In [77]:
#add new columns to the data frame
if 'id_moviedb' not in movies_netflix:
  movies_netflix['id_moviedb'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = str()
if 'vote_average' not in movies_netflix:
  movies_netflix['vote_average'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = 0
if 'media_type' not in movies_netflix:
  movies_netflix['media_type'] = ""

In [84]:
from google.colab import files
import requests
from time import sleep
count = 0

for ind in movies_netflix[0:17769].index: 

    movie_title = movies_netflix['title'][ind]
    year = (str(movies_netflix['year'][ind])[0:4])

    movie_title_part = getFirstPartMovieName(movie_title)

    url = "https://api.themoviedb.org/3/search/multi?api_key="+ apiKey +"&language=en-US&query=" + str(movie_title_part)
    response = requests.get(url)
    results = response.json()

    movie = {}

    print(url)

    if (len(results["results"])) >= 1:
      for item in results["results"]:
        if item["media_type"] != "person":
          if "name" in item:            
            if item["name"] == movie_title:
              movie = item
              break
          if "title" in item: 
            if item["title"] == movie_title:
              movie = item
              break
          if "original_title" in item:             
            if item["original_title"] == movie_title:
              movie = item
              break
          if "release_date" in item:                   
            if period_is_ok(item["release_date"], year):
              movie = item
              break
          if "first_air_date" in item:    
            if period_is_ok(item["first_air_date"], year):
              movie = item
              break
             
    elif (len(results["results"])) == 1:
      movie = results["results"][0]
        
    if movie:
      count = count + 1
      movies_netflix['id_moviedb'][ind] = movie["id"]
      if "genre_ids" in movie:
        movies_netflix['genre_ids'][ind] = movie["genre_ids"]
      if "vote_average" in movie:
        movies_netflix['vote_average'][ind] = movie["vote_average"]
      if "media_type" in movie:
        movies_netflix['media_type'][ind] = movie["media_type"]
      if "popularity" in movie:
        movies_netflix['popularity'][ind] = movie["popularity"]
      print(movie_title + " movie found")
    else:
      print(movie_title + " not found")

    if count == 10:
      # movies_netflix.to_csv('movie_titles.csv', header=True)
      # !cp movie_titles.csv "gdrive/My Drive/"
      count = 0
      sleep(1.5) #due to the limit of calls of the API


https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Dinosaur Planet
Dinosaur Planet movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Isle of Man TT 2004 Review
Isle of Man TT 2004 Review not found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Character
Character movie found


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to b

https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Paula Abdul's Get Up & Dance
Paula Abdul's Get Up & Dance movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=The Rise and Fall of ECW
The Rise and Fall of ECW movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Sick
Sick movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=8 Man
8 Man movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=What the #$*! Do We Know!?
What the #$*! Do We Know!? movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd64f95aaf4e0d&language=en-US&query=Class of Nuke 'Em High 2
Class of Nuke 'Em High 2 movie found
https://api.themoviedb.org/3/search/multi?api_key=defc104635dae32057dd

**Get Movie Details**

**One-hot enconding**

**Pre-Processing**

In [ ]:
#https://developers.themoviedb.org/3/movies/get-movie-details


# movies_netflix.to_csv('movie_titles.csv', header=True)
# !cp movie_titles.csv "gdrive/My Drive/"